In [3]:
from tqdm import tqdm
import os
import sys
import os
os.chdir("..")

from skin_lesion_cad.visualization.utils import simple_im_show, simple_im_show2
from skin_lesion_cad.utils.hair_removal import  HairRemoval

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [5]:
test_images = ['nev00196.jpg', 'nev05913.jpg','nev06446.jpg','nev06467.jpg','mel01439.jpg','nev00294.jpg','ack00317.jpg','bcc00695.jpg','bkl00106.jpg','bkl00140.jpg','bkl00286.jpg','bkl00407.jpg','bkl01086.jpg','bkl01423.jpg','bkl01465.jpg','nev0283.jpg','nev04994.jpg','nev06965.jpg','nev07054.jpg','nev07068.jpg','nev07520.jpg','bcc00064.jpg','bcc00695.jpg','bkl00140.jpg','bkl00286.jpg','bkl00391.jpg','bkl00407.jpg','bkl00936.jpg','bkl01086.jpg','bkl01423.jpg','bkl01465.jpg','mel01953.jpg','mel02047.jpg','mel02558.jpg','mel02697.jpg','nev00170.jpg','nev00373.jpg']

In [6]:
ch1_path = Path('/home/vzalevskyi/uni/MAIA_Semester_3/cad/skin-lesion-cad/data/raw/chall1')
ch2_path = Path('/home/vzalevskyi/uni/MAIA_Semester_3/cad/skin-lesion-cad/data/raw/chall2')

nevus_path = ch1_path/'train/nevus'
random_nevus_img = np.random.choice(list(nevus_path.glob('*')), size=50)

other_path = ch1_path/'train/others'
random_other_img = np.random.choice(list(other_path.glob('*')), size=50)

In [7]:
RAND_IMGS = 15
nevus_path = ch1_path/'train/nevus'
# random_nevus_img = np.random.choice(list(nevus_path.glob('*')), size=RAND_IMGS)

other_path = ch1_path/'train/others'
# random_other_img = np.random.choice(list(other_path.glob('*')), size=RAND_IMGS)

test_images = [nevus_path/t if t[:3] == 'nev' else other_path/t for t in test_images]
test_images = [t for t in test_images if t.exists()]

In [9]:
hr1 = HairRemoval()

for fn in tqdm(list(random_other_img) + list(random_nevus_img) + test_images):
    
    # hair removal needs single channel image, so we transform our images 
    # to grayscale and apply CLAHE for contrast enhancement
    test_image = cv2.imread(str(fn))    
    test_image_bgr = cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR)
    hr_channel_gray = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8)).apply(cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY))

    hair_mask = hr1.get_hair_mask(hr_channel_gray)
    inpainted_img = cv2.inpaint(test_image_bgr, hair_mask, 3, cv2.INPAINT_TELEA)

    fig, axs = plt.subplots(2,2,figsize=(30, 30))
    axs[0, 0].imshow(test_image_bgr)
    axs[0, 0].axis('off')
    axs[1, 0].imshow(inpainted_img)
    axs[1, 0].axis('off')

    axs[0, 1].imshow(hair_mask, cmap='gray')
    axs[0, 1].axis('off')
    axs[1, 1].imshow(hair_mask, cmap='gray')
    axs[1, 1].axis('off')

    
    plt.tight_layout()
    plt.savefig(f"/home/vzalevskyi/uni/MAIA_Semester_3/cad/skin-lesion-cad/notebooks/test_results/{fn.name}1.jpg")
    plt.close(fig)

 26%|██▋       | 36/136 [01:08<03:11,  1.92s/it]


KeyboardInterrupt: 